# Machine Learning in Crowd Modelling and Simulation

In this task we implement a crowd simulation model using the cellular automata approach.
Our implementation can be run either by loading saved scenario files which are based on csv files or by the user detailing their own scenario.

Before moving ahead we import the key packages. It is best to install the required packages as detailed in the
`README.md` file with the help of the provided `requirements.txt` file.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from visualization import visualize_state
from utilities import scenario_loader, scenario_builder

%reload_ext autoreload
%autoreload 2


## Task 1

We will implement the basics of our environment and show the outlook in this section. Here the user has two options

### Option 1: Load scenario
If you wish to load a scenario then type y in the following cell block. A list of available scenarios will be printed. Enter the id number of the scenario you wish to load and this will be loaded into a variable.


### Option 2: Make customer scenario
If you wish to make own scenario then type n in the following cell block. You will be asked for the size of the grid you wish to generate. Enter the row and column numbers. Finally a GUI will pop up. You can make a scenario there. Type in `P` to add a pedestrian, `O` for object and `T` for target. Please note all of these need to be in **upper case**. If you wish for a cell to remain empty you can leave it as it is. Once you are finished simply close the GUI. The scenario will be saved in a variable for further processing. 

In [ ]:
load_scenario_bool = input("Do you want to load a scenario? (y/n)")
    # Case if the user wishes to load a scenario saved in the scenario directory.
if load_scenario_bool == 'y':
    scenario = scenario_loader()
elif load_scenario_bool == 'n':
    scenario = scenario_builder()
else:
    print("Incorrect input. Please try again.")


## Task 2

### Initial Visualization of scenario for Task 2

In [ ]:
# scenario.cells[0,0].cell_type
print(scenario)

In [ ]:
for i in range(len(scenario.pedestrians)):
    print(scenario.pedestrians[i])
    

## Task 3

## Task 4

## Task 5